In [1]:
import pandas as pd
import numpy as np
from typing import Dict

from PIL import Image
from datasets import load_dataset, Dataset, Features, Value

from sklearn.model_selection import train_test_split

# Data

In [2]:
# df = pd.read_json('../data/vlm.jsonl', lines=True)
# df = pd.read_json('../data/vlm.jsonl', lines=True).sample(n=1000, random_state=1)
df = pd.read_json('../data/vlm.jsonl', lines=True).sample(n=3000, random_state=1)

# obj_lst = pd.read_json('../data/categories.jsonl', lines=True)
# categories = np.unique(obj_lst.to_numpy())
# # print(categories)

# id2label = {index: x for index, x in enumerate(categories, start=0)}
# label2id = {v: k for k, v in id2label.items()}

categories = list(filter(lambda x: x != None, pd.DataFrame([i for i in list(df['annotations'].apply(lambda x: [i['caption'] for i in x]))]).to_numpy().flatten()))
num_cat = len(categories)
categories = pd.DataFrame(categories)
categories.drop_duplicates(inplace=True)
categories = categories.values.flatten().tolist()
id2label = {index: x for index, x in enumerate(categories, start=0)}
label2id = {v: k for k, v in id2label.items()}

label2id

{'white and black fighter plane': 0,
 'yellow fighter jet': 1,
 'grey and white light aircraft': 2,
 'grey and white fighter plane': 3,
 'grey and black fighter plane': 4,
 'blue and red commercial aircraft': 5,
 'black and orange drone': 6,
 'black and yellow drone': 7,
 'white and black light aircraft': 8,
 'blue, yellow, and green fighter plane': 9,
 'blue and yellow fighter jet': 10,
 'white and grey helicopter': 11,
 'red and white fighter plane': 12,
 'white missile': 13,
 'white helicopter': 14,
 'blue and green fighter plane': 15,
 'black cargo aircraft': 16,
 'black camouflage fighter jet': 17,
 'yellow, red, and grey helicopter': 18,
 'red and black drone': 19,
 'blue and white commercial aircraft': 20,
 'grey and red fighter jet': 21,
 'black fighter plane': 22,
 'white and black helicopter': 23,
 'silver and blue fighter plane': 24,
 'white cargo aircraft': 25,
 'grey fighter jet': 26,
 'white, blue, and red commercial aircraft': 27,
 'white and black drone': 28,
 'green an

In [3]:
df["image_id"] = df.index

df['width'] = df['image'].apply(lambda x: Image.open('../data/images/' + x).size)
df['height'] = df['width'].apply(lambda x: x[1])
df['width'] = df['width'].apply(lambda x: x[0])

df['img'] = df['image'].apply(lambda x: Image.open('../data/images/' +x))

df['area'] = df['annotations'].apply(lambda x: {"area":[i['bbox'][2] * i['bbox'][3] for i in x]})
df['bbox'] = df['annotations'].apply(lambda x: {"bbox":[i['bbox'] for i in x]})
# df['category'] = df['annotations'].apply(lambda x: {'category':[label2id[i['caption']] for i in x]})
def check_str(caption, categories, label2id):
	for cat in categories:
		if cat in caption:
			return label2id[cat]

df['category'] = df['annotations'].apply(lambda x: {'category':[check_str(i['caption'], categories, label2id) for i in x]})
df

,image,annotations,image_id,width,height,img,area,bbox,category
5074,image_5074.jpg,"[{'caption': 'white and black fighter plane', ...",5074,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [2640, 2688, 1872, 1872, 2240, 6272, ...","{'bbox': [[1020, 128, 60, 44], [812, 184, 48, ...","{'category': [0, 1, 2, 3, 4, 5, 6, 7]}"
2711,image_2711.jpg,"[{'caption': 'grey and white light aircraft', ...",2711,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [2048, 2496, 1024, 1280, 3040]}","{'bbox': [[288, 240, 64, 32], [1072, 512, 48, ...","{'category': [2, 8, 9, 1, 11]}"
3670,image_3670.jpg,"[{'caption': 'red and white fighter plane', 'b...",3670,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [960, 3072, 1408, 1536]}","{'bbox': [[1180, 240, 40, 24], [1024, 368, 64,...","{'category': [12, 13, 14, 15]}"
2377,image_2377.jpg,"[{'caption': 'black cargo aircraft', 'bbox': [...",2377,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [1408, 2448, 1936, 1792, 1872, 5632, ...","{'bbox': [[428, 344, 44, 32], [560, 192, 68, 3...","{'category': [16, 6, 17, 18, 19, 20, 21]}"
4195,image_4195.jpg,"[{'caption': 'black fighter plane', 'bbox': [7...",4195,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [1456, 2688, 2304, 3072, 3840, 2160, ...","{'bbox': [[716, 384, 52, 28], [784, 288, 56, 4...","{'category': [22, 8, 23, 24, 25, 26, 27]}"
...,...,...,...,...,...,...,...,...,...
1357,image_1357.jpg,"[{'caption': 'blue, yellow, and white cargo ai...",1357,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [3808, 960, 2176, 6656, 3168, 2912]}","{'bbox': [[1140, 264, 68, 56], [720, 168, 40, ...","{'category': [25, 105, 23, 50, 35, 4]}"
3892,image_3892.jpg,"[{'caption': 'yellow, red, and blue fighter pl...",3892,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [1152, 3520, 1056, 2128, 6160, 2016]}","{'bbox': [[1000, 136, 36, 32], [336, 176, 80, ...","{'category': [47, 11, 12, 45, 73, 15]}"
3258,image_3258.jpg,"[{'caption': 'grey and black helicopter', 'bbo...",3258,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [1728, 2304, 5040, 1440, 4080]}","{'bbox': [[1080, 112, 48, 36], [424, 344, 48, ...","{'category': [38, 41, 79, 51, 12]}"
3507,image_3507.jpg,"[{'caption': 'grey and yellow fighter plane', ...",3507,1520,870,<PIL.JpegImagePlugin.JpegImageFile image mode=...,"{'area': [1536, 1440, 896]}","{'bbox': [[432, 352, 48, 32], [664, 216, 36, 4...","{'category': [51, 63, 62]}"


In [4]:
df = df.drop(columns=['annotations'])

counter = 0
lst = []
for i in df['category']:
	size = len(i['category'])
	each = list(np.arange(counter, counter + size))
	lst.append({'id':each})
	counter += size
df['id'] = lst

df_vlm = pd.DataFrame({
	"file_name" : df['image'],
	"image_id" : df['image_id'],
	"image" : df['img'],
	"width" : df['width'],
	"height": df['height'],
	"objects": df.apply(lambda x: {'id': x['id']['id'], 'area': x['area']['area'],'bbox': x['bbox']['bbox'], 'category': x['category']['category']}, axis=1)
}).sort_index(axis=0)
# df_vlm.to_csv('meta_data.csv', index=False)
df_vlm

,file_name,image_id,image,width,height,objects
5074,image_5074.jpg,5074,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [0, 1, 2, 3, 4, 5, 6, 7], 'area': [2640..."
2711,image_2711.jpg,2711,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [8, 9, 10, 11, 12], 'area': [2048, 2496..."
3670,image_3670.jpg,3670,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [13, 14, 15, 16], 'area': [960, 3072, 1..."
2377,image_2377.jpg,2377,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [17, 18, 19, 20, 21, 22, 23], 'area': [..."
4195,image_4195.jpg,4195,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [24, 25, 26, 27, 28, 29, 30], 'area': [..."
...,...,...,...,...,...,...
1357,image_1357.jpg,1357,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16302, 16303, 16304, 16305, 16306, 163..."
3892,image_3892.jpg,3892,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16308, 16309, 16310, 16311, 16312, 163..."
3258,image_3258.jpg,3258,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16314, 16315, 16316, 16317, 16318], 'a..."
3507,image_3507.jpg,3507,<PIL.JpegImagePlugin.JpegImageFile image mode=...,1520,870,"{'id': [16319, 16320, 16321], 'area': [1536, 1..."


# Split Dataset

In [5]:
df_vlm_csv = df_vlm.drop(columns=['image'])
df_vlm_csv.to_csv('../data/images/metadata.csv', index=False)
# pd.DataFrame({
# 	'name':df_vlm['file_name'],
# 	'img':df_vlm['image']
# }).apply(lambda x: x[1].save('../vlm-images2/images/' + x[0]), axis=1)